📄 .gitattributes
📄 .gitignore
📄 LICENSE
📄 README (3).md
📄 README.md
📁 backend
    📁 api
        📄 upload.py
    📄 config.py
    📄 main.py
    📁 models
        📄 skin_disease_model.h5
    📁 services
        📄 image_classifier.py
        📄 symptom_to_disease.py
    📁 utils
        📄 filtering_with_ner.py
        📄 image_preprocessing.py
        📄 text_cleaning.py
📁 data
    📁 Vector
        📁 symptom_faiss_db
            📄 index.faiss
            📄 index.pkl
    📄 labels.json
    📄 mayo_diseases.csv
    📄 test_symptom_cases.csv
📁 evaluation
    📄 evaluate_rag_symptom.py
📁 frontend
    📄 index.html
📄 logo.png
📁 notebook
    📄 llm.ipynb
    📄 skin_disease_prediction.ipynb
    📄 web_scrapping.ipynb
📄 requirements.txt
📁 scripts
    📄 model.py
    📁 scrapers
        📄 main.py
        📄 web_scraper2.py
        📄 web_scrapers.py
    📄 symptoms_to_vectordb.py
📄 transfaer.ipynb


In [8]:
import pandas as pd

df = pd.read_excel("ti.xls")

In [9]:
df.head()

,Reg No,StudentName,Platform,Reporting Time,Venue,Instructions
0,12206253,Korumilli Varun SaiNadh,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
1,12220486,Asmi Raj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
2,12222321,Thiluck Vardhan Vemula,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
3,12220695,Ritvik Bharti,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
4,12210943,Amrit Raj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Reg No          948 non-null    int64 
 1   StudentName     948 non-null    object
 2   Platform        948 non-null    object
 3   Reporting Time  948 non-null    object
 4   Venue           948 non-null    object
 5   Instructions    948 non-null    object
dtypes: int64(1), object(5)
memory usage: 44.6+ KB


In [22]:
from datetime import time

df_filtered = df[
    (df["Reporting Time"] == time(9, 30)) & 
    (df["Venue"].str.strip() == "33-501")
]


In [23]:
len(df_filtered)

60

In [24]:
df_filtered

,Reg No,StudentName,Platform,Reporting Time,Venue,Instructions
0,12206253,Korumilli Varun SaiNadh,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
1,12220486,Asmi Raj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
2,12222321,Thiluck Vardhan Vemula,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
3,12220695,Ritvik Bharti,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
4,12210943,Amrit Raj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
5,12204079,Harish Baranwal,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
6,12216999,Tushar Bhardwaj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
7,12223546,Sunny Kumar,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
8,12217365,Anurag Chauhan,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...
9,12216901,Amulya Raj,ImNeo,09:30:00,33-501,1. Test will be conducted on NEO Browser.\n2. ...


In [21]:
print(df["Reporting Time"].unique())


[datetime.time(9, 30) datetime.time(12, 30)]


In [26]:
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
